# 🏀 NBA Predictor - Cloud Training (FIXED)

## Steps:
1. Upload your files
2. Run training
3. Download models

**Time:** 20-30 minutes with GPU

In [ ]:
# ============================================================
# STEP 1: Upload Your Data Files
# ============================================================
# Upload 2 files:
# 1. PlayerStatistics.csv.zip (39.5 MB - compressed)
# 2. priors_data.zip (Basketball Reference stats)

from google.colab import files
import os

print("📤 Upload PlayerStatistics.csv.zip AND priors_data.zip:")
uploaded = files.upload()

# Extract both files
print("\n📦 Extracting files...")
if os.path.exists('PlayerStatistics.csv.zip'):
    !unzip -q PlayerStatistics.csv.zip
    !rm PlayerStatistics.csv.zip
    print("✅ PlayerStatistics.csv extracted")

if os.path.exists('priors_data.zip'):
    !unzip -q priors_data.zip
    print("✅ priors_data extracted")

print("\n✅ All files uploaded and ready!")

In [ ]:
# ============================================================
# STEP 2: Setup & Train
# ============================================================

print("📦 Installing packages...")
!pip install -q nba-api kagglehub pytorch-tabnet lightgbm scikit-learn pandas numpy tqdm

print("\n📥 Downloading code...")
import os
os.chdir('/content')
!git clone https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')

print("\n📍 Code version:")
!git log -1 --oneline

# Check GPU
import torch
print(f"\n🎮 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")

# VERIFY FIX IS PRESENT
print("\n🔍 Verifying player data fix...")
import pandas as pd
import numpy as np

# Test the fix on a sample
ps_sample = pd.read_csv('/content/PlayerStatistics.csv', nrows=10000, low_memory=False)
date_col = [c for c in ps_sample.columns if 'date' in c.lower()][0]
ps_sample[date_col] = pd.to_datetime(ps_sample[date_col], errors='coerce')

# Use exact logic from train_auto.py
def _season_from_date(dt):
    if pd.api.types.is_datetime64_any_dtype(dt):
        d = dt
    else:
        d = pd.to_datetime(dt, errors="coerce", utc=False)
    y = d.dt.year
    m = d.dt.month
    return np.where(m >= 8, y + 1, y)

# Test filtering
temp_seasons = pd.Series(_season_from_date(ps_sample[date_col]))
ps_sample['_temp_season'] = temp_seasons.fillna(-1).astype(int)
window_test = {2007, 2008, 2009, 2010, 2011, 2006, 2012}
filtered = ps_sample[ps_sample['_temp_season'].isin(window_test)]

if len(filtered) == 0:
    print("❌ WARNING: Fix not working! Player data will be 0 rows!")
    print("   Check that code was cloned correctly.")
else:
    print(f"✅ Fix verified! Sample filtering: {len(ps_sample):,} → {len(filtered):,} rows")

print("\n🚀 Starting training (20-30 min)...\n")
!python3 train_auto.py \
    --priors-dataset /content/priors_data \
    --player-csv /content/PlayerStatistics.csv \
    --verbose \
    --fresh \
    --neural-device gpu \
    --neural-epochs 50

print("\n✅ TRAINING COMPLETE!")

In [ ]:
# ============================================================
# STEP 3: Download Models
# ============================================================

from google.colab import files

print("📦 Packaging models...")
!zip -q -r nba_models_trained.zip models/ model_cache/

print("💾 Downloading...")
files.download('nba_models_trained.zip')

print("\n✅ Done! Extract nba_models_trained.zip to your local nba_predictor folder.")

---

## ❓ Troubleshooting

### "Loaded 0 player-games for window"
- Make sure you uploaded **PlayerStatistics.csv.zip** (not the uncompressed version)
- Verify file is 39.5 MB compressed

### "No GPU available"
- Runtime → Change runtime type → GPU (T4 or L4)

### "Out of memory"
- Runtime → Restart runtime
- Re-run from Step 1

---

**Version:** 2.2 (Simplified, Fixed)

**What's included:**
- Game models (moneyline, spread)
- Player models (points, rebounds, assists, 3PM, minutes)
- Ensemble (Ridge + Elo + Four Factors + LightGBM)
- Neural hybrid (TabNet + LightGBM)
- 20+ years of historical data
- Basketball Reference priors (~68 features)

**Expected output:**
```
Training window 1/5: 2002-2006
  • Loaded 245,892 player-games for window  ✅
```